## 2. Preprocessing
`whisky_reviews.csv` 데이터 전처리

<img style="float: right;" src="../img/logo.png" width="120"><br>

<div style="text-align: right"> <b>Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial issue : 2025.11.16 </div>
<div style="text-align: right"> last update : 2025.11.16 </div>

개정 이력  
- `2025.11.16` : 노트북 초기 생성 

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
import pandas as pd
import numpy as np
from rag_pkg.utils.path import RAW_DATA_PATH, INTERMEDIATE_DATA_PATH, PROCESSED_DATA_PATH

In [3]:
review_data_path = RAW_DATA_PATH / "whisky_reviews.csv"

reviews = pd.read_csv(review_data_path)

In [4]:
# 데이터 기본 정보 확인
print("데이터 Shape:", reviews.shape)
print("\n결측치 확인:")
print(reviews.isnull().sum())
print("\n데이터 타입:")
print(reviews.dtypes)
print("\n상위 5개 행:")
reviews.head()

데이터 Shape: (985, 9)

결측치 확인:
Whisky Name         0
Link                0
Tags              615
Nose Score        531
Nose Comment      270
Taste Score       531
Taste Comment     269
Finish Score      531
Finish Comment    270
dtype: int64

데이터 타입:
Whisky Name        object
Link               object
Tags               object
Nose Score        float64
Nose Comment       object
Taste Score       float64
Taste Comment      object
Finish Score      float64
Finish Comment     object
dtype: object

상위 5개 행:


,Whisky Name,Link,Tags,Nose Score,Nose Comment,Taste Score,Taste Comment,Finish Score,Finish Comment
0,Springbank1966,https://www.whiskybase.com/whiskies/whisky/132...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Springbank10-year-old,https://www.whiskybase.com/whiskies/whisky/416...,Green-House,94.0,The nose is full of aromatic power. We still h...,96.0,"On the palate it is surprisingly fresh and ""al...",95.0,"Long finish on liquorice, camphor, smoke, ash ..."
2,Ardbeg1967 Kb,https://www.whiskybase.com/whiskies/whisky/230...,NaN,NaN,"Peat, nuts, celery, well integrated, leather, ...",NaN,"Leather, shoe polish, sherry, mints, cigar, sm...",NaN,"Banana, smoke, citrus, black tea, long and nutty"
3,Springbank35-year-old,https://www.whiskybase.com/whiskies/whisky/110...,"Chocolate,Dried Fruit,Fresh Fruit,New Wood,Oil...",100.0,"Fresh, rich, intensive, complex, fruity, plums...",99.0,"Oh yes, smooth, warm, sweet oak wood, little v...",98.0,"Long, warm, very smooth oak lingering - nevere..."
4,Springbank1966,https://www.whiskybase.com/whiskies/whisky/143...,NaN,NaN,Like a chimera of Genting King and Honey Rum. ...,NaN,"Oceanic fino Shirley. Rhubarb, ginkgo fruit, n...",NaN,"Long-term, coffee, salt, fino, black pepper, c..."


### 1. 결측치 처리(필터링) 전략

RAG 시스템의 검색 품질을 위해 다음과 같은 결측치 처리 전략 수행

1. **필수 정보**: Whisky Name, Link는 반드시 있어야 함
2. **RAG 컨텍스트 품질**: Nose/Taste/Finish 중 적어도 **2개 이상의 Comment**가 있어야 의미 있는 추천 가능
3. **Score는 선택적**: Comment가 충분하다면 Score 누락은 허용
4. **보간 전략**: 
   - Comment는 보간 불가 (텍스트 정보는 임의 생성 불가)
   - Score는 평균값 보간 가능하지만, RAG에서는 Comment가 더 중요

In [5]:
def filter_valid_reviews(df: pd.DataFrame, min_comments: int = 2) -> pd.DataFrame:
    """
    RAG에 적합한 유효한 위스키 리뷰만 필터링합니다.
    
    Parameters:
    -----------
    df : pd.DataFrame
        원본 위스키 리뷰 데이터프레임
    min_comments : int, default=2
        최소 필요한 Comment 개수 (Nose/Taste/Finish 중)
    
    Returns:
    --------
    pd.DataFrame
        필터링된 데이터프레임
    
    Filtering Logic:
    ----------------
    1. Whisky Name과 Link는 필수
    2. Nose Comment, Taste Comment, Finish Comment 중 최소 min_comments개 이상 존재
    3. 빈 문자열('')도 결측치로 간주
    """
    df_filtered = df.copy()
    
    # 1. 필수 필드 체크: Whisky Name, Link
    essential_mask = (
        df_filtered['Whisky Name'].notna() & 
        (df_filtered['Whisky Name'].str.strip() != '') &
        df_filtered['Link'].notna() & 
        (df_filtered['Link'].str.strip() != '')
    )
    
    # 2. Comment 필드 유효성 체크
    comment_cols = ['Nose Comment', 'Taste Comment', 'Finish Comment']
    
    # 각 Comment가 유효한지 체크 (NaN이 아니고, 빈 문자열이 아님)
    valid_comments_count = 0
    for col in comment_cols:
        valid_comments_count += (
            df_filtered[col].notna() & 
            (df_filtered[col].str.strip() != '')
        ).astype(int)
    
    # 최소 min_comments개 이상의 Comment가 있는 행만 선택
    comments_mask = valid_comments_count >= min_comments
    
    # 최종 필터링
    final_mask = essential_mask & comments_mask
    
    df_result = df_filtered[final_mask].copy()
    
    # 인덱스 리셋
    df_result = df_result.reset_index(drop=True)
    
    return df_result

In [6]:
# 필터링 적용
filtered_reviews = filter_valid_reviews(reviews, min_comments=2)

print(f"원본 데이터: {len(reviews)}개")
print(f"필터링 후: {len(filtered_reviews)}개")
print(f"제거된 행: {len(reviews) - len(filtered_reviews)}개 ({(len(reviews) - len(filtered_reviews)) / len(reviews) * 100:.1f}%)")
print("\n필터링 후 결측치:")
print(filtered_reviews.isnull().sum())

원본 데이터: 985개
필터링 후: 716개
제거된 행: 269개 (27.3%)

필터링 후 결측치:
Whisky Name         0
Link                0
Tags              418
Nose Score        330
Nose Comment        1
Taste Score       330
Taste Comment       0
Finish Score      330
Finish Comment      1
dtype: int64


### 3 RAG를 위한 텍스트 결합 함수

RAG 시스템에서 검색될 텍스트 청크를 생성   
각 위스키의 정보를 구조화된 형태로 결합하여 임베딩에 적합한 형태로 만듬

In [7]:
def create_document_text(row: pd.Series) -> str:
    """
    각 위스키 리뷰를 RAG용 단일 문서 텍스트로 결합합니다.
    
    Parameters:
    -----------
    row : pd.Series
        위스키 리뷰 데이터의 한 행
    
    Returns:
    --------
    str
        임베딩에 사용할 구조화된 텍스트
    
    Format:
    -------
    위스키 이름: {name}
    태그: {tags}
    
    향(Nose): {nose_comment}
    
    맛(Taste): {taste_comment}
    
    피니쉬(Finish): {finish_comment}
    """
    parts = [f"위스키 이름: {row['Whisky Name']}"]
    
    # Tags가 있으면 추가
    if pd.notna(row['Tags']) and row['Tags'].strip():
        parts.append(f"태그: {row['Tags']}")
    
    parts.append("")  # 빈 줄 추가
    
    # Nose Comment
    if pd.notna(row['Nose Comment']) and row['Nose Comment'].strip():
        nose_text = f"향(Nose)"
        if pd.notna(row['Nose Score']) and str(row['Nose Score']).strip():
            nose_text += f" [점수: {row['Nose Score']}]"
        nose_text += f": {row['Nose Comment']}"
        parts.append(nose_text)
        parts.append("")
    
    # Taste Comment
    if pd.notna(row['Taste Comment']) and row['Taste Comment'].strip():
        taste_text = f"맛(Taste)"
        if pd.notna(row['Taste Score']) and str(row['Taste Score']).strip():
            taste_text += f" [점수: {row['Taste Score']}]"
        taste_text += f": {row['Taste Comment']}"
        parts.append(taste_text)
        parts.append("")
    
    # Finish Comment
    if pd.notna(row['Finish Comment']) and row['Finish Comment'].strip():
        finish_text = f"피니쉬(Finish)"
        if pd.notna(row['Finish Score']) and str(row['Finish Score']).strip():
            finish_text += f" [점수: {row['Finish Score']}]"
        finish_text += f": {row['Finish Comment']}"
        parts.append(finish_text)
    
    return "\n".join(parts)

In [8]:
# 문서 텍스트 생성
filtered_reviews['document_text'] = filtered_reviews.apply(create_document_text, axis=1)

# 예시 출력
print("=" * 80)
print("첫 번째 위스키 문서 예시:")
print("=" * 80)
print(filtered_reviews['document_text'].iloc[0])
print("\n" + "=" * 80)
print("두 번째 위스키 문서 예시:")
print("=" * 80)
print(filtered_reviews['document_text'].iloc[1])

첫 번째 위스키 문서 예시:
위스키 이름: Springbank10-year-old
태그: Green-House

향(Nose) [점수: 94.0]: The nose is full of aromatic power. We still have a little touch of solvent.\nLeather, orange peel, star anise.\nPlum, prunes, dates, figs, clementine, passion fruit peel.\nOld dry wood, dust, old book.\nWe have aromas of old rum, Demerara, and almost a little cane sugar.

맛(Taste) [점수: 96.0]: On the palate it is surprisingly fresh and "almost" light.\nFresh apricot, pineapple, passion fruit, guava, papaya.\nIt is very tropical.\nBut the dominant remains woody, with pretty spices.\nCloves, anise, pepper, bitter chocolate, cinnamon, nutmeg. They are all there.\nWe have fresh mint, some dry aromatic herbs.\nA little barbecue charcoal, smoke.

피니쉬(Finish) [점수: 95.0]: Long finish on liquorice, camphor, smoke, ash and barbecue, light peat.\nPepper, cloves, fresh mint.\nIt's long, comforting, it feels like you're at the edge of the fireplace.

두 번째 위스키 문서 예시:
위스키 이름: Ardbeg1967 Kb

향(Nose): Peat, nuts, celery,

### 4 전처리된 데이터 저장

필터링 및 문서화된 데이터를 중간 데이터로 저장

In [9]:
# 전처리된 데이터 저장
output_path = INTERMEDIATE_DATA_PATH / "filtered_whisky_reviews.csv"
filtered_reviews.to_csv(output_path, index=False)

print(f"전처리된 데이터 저장 완료: {output_path}")
print(f"총 {len(filtered_reviews)}개의 유효한 위스키 리뷰")

전처리된 데이터 저장 완료: /mnt/d/project/rag-assignment/data/intermediate/filtered_whisky_reviews.csv
총 716개의 유효한 위스키 리뷰


### 5. 데이터 품질 검증
필터링된 데이터의 품질을 확인

In [10]:
# Comment 필드별 유효 데이터 개수
comment_cols = ['Nose Comment', 'Taste Comment', 'Finish Comment']

print("각 Comment 필드의 유효 데이터 개수:")
for col in comment_cols:
    valid_count = (filtered_reviews[col].notna() & (filtered_reviews[col].str.strip() != '')).sum()
    print(f"  {col}: {valid_count}/{len(filtered_reviews)} ({valid_count/len(filtered_reviews)*100:.1f}%)")

print("\n평균 문서 텍스트 길이:")
avg_length = filtered_reviews['document_text'].str.len().mean()
print(f"  {avg_length:.0f} characters")

print("\n문서 텍스트 길이 분포:")
print(filtered_reviews['document_text'].str.len().describe())

각 Comment 필드의 유효 데이터 개수:
  Nose Comment: 715/716 (99.9%)
  Taste Comment: 716/716 (100.0%)
  Finish Comment: 715/716 (99.9%)

평균 문서 텍스트 길이:
  685 characters

문서 텍스트 길이 분포:
count     716.000000
mean      684.787709
std       349.593762
min        77.000000
25%       459.750000
50%       602.500000
75%       852.500000
max      2862.000000
Name: document_text, dtype: float64


### 6. 함수 import

In [11]:
# 함수 결과 동일성 검증
print("=" * 80)
print("함수 결과 동일성 검증")
print("=" * 80)

# 1. filter_valid_reviews 함수 테스트
print("\n1. filter_valid_reviews 함수 검증:")
print("-" * 80)

# 노트북에서 정의한 함수로 필터링 (이미 실행됨)
notebook_filtered = filtered_reviews.copy()

# import한 함수로 필터링
from rag_pkg.module.preprocess import filter_valid_reviews as imported_filter
from rag_pkg.module.preprocess import create_document_text as imported_create_doc

imported_filtered = imported_filter(reviews, min_comments=2)

# Shape 비교
print(f"  Shape 일치: {notebook_filtered.shape == imported_filtered.shape}")
print(f"    - Notebook 함수: {notebook_filtered.shape}")
print(f"    - Import 함수: {imported_filtered.shape}")

# 데이터프레임 비교 (document_text 컬럼 제외)
cols_to_compare = [col for col in notebook_filtered.columns if col != 'document_text']
df_equal = notebook_filtered[cols_to_compare].equals(imported_filtered[cols_to_compare])
print(f"\n  DataFrame 일치 (document_text 제외): {df_equal}")

# 2. create_document_text 함수 테스트
print("\n2. create_document_text 함수 검증:")
print("-" * 80)

# 샘플 행으로 테스트 (첫 5개 행)
all_texts_equal = True
for idx in range(min(5, len(imported_filtered))):
    notebook_text = create_document_text(imported_filtered.iloc[idx])
    imported_text = imported_create_doc(imported_filtered.iloc[idx])
    
    texts_equal = notebook_text == imported_text
    all_texts_equal = all_texts_equal and texts_equal
    
    if not texts_equal:
        print(f"  [행 {idx}] 불일치 발견!")
        print(f"    Notebook: {notebook_text[:100]}...")
        print(f"    Import: {imported_text[:100]}...")
    else:
        print(f"  [행 {idx}] ✓ 일치")

print(f"\n  전체 샘플 텍스트 일치: {all_texts_equal}")

# 3. 전체 파이프라인 검증
print("\n3. 전체 파이프라인 검증:")
print("-" * 80)

# import한 함수로 전체 파이프라인 실행
imported_filtered['document_text'] = imported_filtered.apply(imported_create_doc, axis=1)

# document_text 컬럼도 포함하여 비교
full_pipeline_equal = (
    notebook_filtered.shape == imported_filtered.shape and
    all(notebook_filtered['Whisky Name'] == imported_filtered['Whisky Name']) and
    all(notebook_filtered['document_text'] == imported_filtered['document_text'])
)

print(f"  전체 파이프라인 결과 일치: {full_pipeline_equal}")

# 4. 최종 검증 결과
print("\n" + "=" * 80)
print("최종 검증 결과:")
print("=" * 80)

if df_equal and all_texts_equal and full_pipeline_equal:
    print("✅ 모든 함수가 정상적으로 일치합니다!")
    print("   노트북 정의 함수와 import한 함수의 결과가 동일합니다.")
else:
    print("❌ 불일치가 발견되었습니다!")
    print(f"   - DataFrame 일치: {df_equal}")
    print(f"   - 텍스트 생성 일치: {all_texts_equal}")
    print(f"   - 전체 파이프라인 일치: {full_pipeline_equal}")

함수 결과 동일성 검증

1. filter_valid_reviews 함수 검증:
--------------------------------------------------------------------------------
  Shape 일치: False
    - Notebook 함수: (716, 10)
    - Import 함수: (716, 9)

  DataFrame 일치 (document_text 제외): True

2. create_document_text 함수 검증:
--------------------------------------------------------------------------------
  [행 0] ✓ 일치
  [행 1] ✓ 일치
  [행 2] ✓ 일치
  [행 3] ✓ 일치
  [행 4] ✓ 일치

  전체 샘플 텍스트 일치: True

3. 전체 파이프라인 검증:
--------------------------------------------------------------------------------
  전체 파이프라인 결과 일치: True

최종 검증 결과:
✅ 모든 함수가 정상적으로 일치합니다!
   노트북 정의 함수와 import한 함수의 결과가 동일합니다.
